<a href="https://colab.research.google.com/github/NathanUrinovsky/Final-Project/blob/main/building_image_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connecting Google Drive to Colab
#this is the code for training only
#this project can be run only from Google Colab, 
#and you must have the dataset in your drive in order to train the model
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install keras-retinanet

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
from tensorflow import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import pathlib
import time

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras

setup_gpu(gpu)

In [ ]:
data_folder = os.getcwd() + "/data/"
!mkdir {data_folder}

In [ ]:
# import packages
import os
import glob
import pandas as pd
import argparse
import xml.etree.ElementTree as ET
import random
from shutil import copyfile
import csv

training_images = pathlib.Path('/content/gdrive/My Drive/MachineLearning/training_data/images/')
labels = pathlib.Path('/content/gdrive/My Drive/MachineLearning/training_data/labels/')
training_images = os.listdir(training_images)
training_images_list = []
for file in training_images:
  copyfile('/content/gdrive/My Drive/MachineLearning/training_data/images/' + file, data_folder + file)
  training_images_list.append(data_folder + file)


#labels = os.listdir(labels)
test_images = []
train_csv = "/content/data/train.csv"
test_csv = "/content/data/test.csv"
classes_csv = "/content/data/classes.csv"

In [ ]:
dataset = [
("train", training_images, train_csv)
]

In [ ]:
# initialize set of classes and annotations
classes = set()

# loop over train and test datasets
for (dType, imagePaths, outputCSV) in dataset:

    annotations = []
    # load the contents
    print("[INFO] creating '{}' set ...".format(dType))
    print("[INFO] {} total images in '{}'".format(len(imagePaths), dType))

    # open the output CSV file
    with open(outputCSV, "w") as o:
        writer = csv.writer(o)

        # loop over image paths
        for imagePath in imagePaths:
            # generate corresponding annotation path
            fname = os.path.basename(imagePath).split(".")[0]
            annotPath = os.path.join(labels, fname + ".xml")

            tree = ET.parse(annotPath)
            root = tree.getroot()

            # loop over all object elements
            for elem in root:
                if elem.tag == "object":
                    obj_name = None
                    coords = []
                    for subelem in elem:
                        # extract all label and bounding box coordinates
                        if subelem.tag == "name":
                            obj_name = subelem.text
                        if subelem.tag == "bndbox":
                            for subsubelem in subelem:
                                coords.append(int(float((subsubelem.text))))
                            xMin = coords[0]
                            yMin = coords[1]
                            xMax = coords[2]
                            yMax = coords[3]

                    # truncate any bounding box corrdinates that fall outside
                    # image boundaries
                    xMin = max(0, xMin)
                    yMin = max(0, yMin)
                    xMax = max(0, xMax)
                    yMax = max(0, yMax)

                    # ignore bounding boxes where min values are larger than
                    # max values and vice-versa (annotation errors)
                    if xMin >= xMax or yMin >= yMax:
                        continue
                    elif xMax <= xMin or yMax <= yMin:
                        continue

                    # gather image path, coordinates, and class for each object on image
                    item = [imagePath] + coords + [obj_name]
                    # create list of information of all objects in image
                    annotations.append(item)
                    # write to outputCSV
                    writer.writerows(annotations)
                    # update set of unique class labels
                    classes.add(obj_name)

    # write the annotations to file
    print("[INFO] writing {} annotations ...".format(dType))
    print("[INFO] total {} annotations".format(len(annotations)))
    with open(outputCSV, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(annotations)
    print("[INFO] {}.csv completed".format(dType))

# write the classes to file
print("[INFO] writing classes ...")
with open(classes_csv, 'w') as f:
    for i, line in enumerate(classes):
      f.write('{},{}\n'.format(line,i))
print("[INFO] classes.csv completed")

print("[FINAL] Task completed!")

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/keras_retinanet/bin/train.py \
--freeze-backbone \
--random-transform \
--weighted-average \
--batch-size 6 \
--steps 50 \
--epochs 30 csv '/content/data/train.csv' '/content/data/classes.csv'